# Anime Recommendation System

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import pickle
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df= pd.read_csv('Anime_dataset_2023_1.csv')

In [3]:
df.head()

,Unnamed: 0,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Aired,Status,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Members,Image URL
0,0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Finished Airing,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",Finished Airing,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,2,6,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Finished Airing,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",Finished Airing,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,4,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",Finished Airing,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [4]:
df.columns

Index(['Unnamed: 0', 'anime_id', 'Name', 'Score', 'Genres', 'Synopsis', 'Type',
       'Episodes', 'Aired', 'Status', 'Studios', 'Source', 'Duration',
       'Rating', 'Rank', 'Popularity', 'Favorites', 'Members', 'Image URL'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24905 non-null  int64 
 1   anime_id    24905 non-null  int64 
 2   Name        24905 non-null  object
 3   Score       24905 non-null  object
 4   Genres      24905 non-null  object
 5   Synopsis    24905 non-null  object
 6   Type        24905 non-null  object
 7   Episodes    24905 non-null  object
 8   Aired       24905 non-null  object
 9   Status      24905 non-null  object
 10  Studios     24905 non-null  object
 11  Source      24905 non-null  object
 12  Duration    24905 non-null  object
 13  Rating      24905 non-null  object
 14  Rank        24905 non-null  object
 15  Popularity  24905 non-null  int64 
 16  Favorites   24905 non-null  int64 
 17  Members     24905 non-null  int64 
 18  Image URL   24905 non-null  object
dtypes: int64(5), object(14)
memory usage: 3.6+ MB


In [6]:
df['Type'].unique()

array(['TV', 'Movie', 'OVA', 'Special', 'ONA', 'Music', 'UNKNOWN'],
      dtype=object)

In [7]:
df['Genres'].unique()

array(['Action, Award Winning, Sci-Fi', 'Action, Sci-Fi',
       'Action, Adventure, Sci-Fi', ..., 'Girls Love, Sci-Fi',
       'Action, Comedy, Drama, Fantasy, Romance, Ecchi',
       'Action, Adventure, Fantasy, Horror'], dtype=object)

In [23]:
Genress = set()

for gen in df['Genres']:
    genre = gen.split(',')

    for gens in genre :
        Genress.add(gens)
#Genress = set(reversed(list(Genress)))
Genress = list(Genress)
Genress.reverse()
print(Genress)

['Adventure', 'Action', ' Mystery', ' Sci-Fi', 'Ecchi', 'Fantasy', ' Comedy', 'Supernatural', 'Girls Love', 'Sports', 'Romance', ' Horror', ' Fantasy', ' Girls Love', 'Hentai', 'Horror', 'Award Winning', 'Erotica', ' Adventure', ' Slice of Life', ' Drama', 'Gourmet', ' Romance', 'Sci-Fi', 'Drama', ' Boys Love', ' Gourmet', 'Boys Love', ' Suspense', ' Award Winning', ' Sports', 'Slice of Life', 'UNKNOWN', ' Supernatural', 'Avant Garde', ' Hentai', ' Erotica', 'Suspense', ' Avant Garde', ' Ecchi', 'Comedy', 'Mystery']


In [24]:
df['Rating'].unique()

array(['R - 17+ (violence & profanity)', 'PG-13 - Teens 13 or older',
       'PG - Children', 'R+ - Mild Nudity', 'G - All Ages', 'Rx - Hentai',
       'UNKNOWN'], dtype=object)

In [25]:
df['Name'].unique()

array(['Cowboy Bebop', 'Cowboy Bebop: Tengoku no Tobira', 'Trigun', ...,
       'Di Yi Xulie', 'Bokura no Saishuu Sensou', 'Shijuuku Nichi'],
      dtype=object)